In [1]:
!uv pip install mlflow

Using Python 3.11.12 environment at: /usr
Resolved 74 packages in 1.79s
⠙ Preparing packages... (0/13)
⠙ Preparing packages... (0/13)
⠙ Preparing packages... (0/13)
graphql-relay ------------------------------ 14.86 KiB/16.54 KiB
⠙ Preparing packages... (0/13)
graphql-relay ------------------------------ 14.86 KiB/16.54 KiB
mako       ------------------------------     0 B/76.67 KiB
⠙ Preparing packages... (0/13)
graphql-relay ------------------------------ 14.86 KiB/16.54 KiB
mako       ------------------------------     0 B/76.67 KiB
gunicorn   ------------------------------     0 B/83.04 KiB
⠙ Preparing packages... (0/13)
graphql-relay ------------------------------ 14.86 KiB/16.54 KiB
starlette  ------------------------------     0 B/70.35 KiB
mako       ------------------------------     0 B/76.67 KiB
gunicorn   ------------------------------     0 B/83.04 KiB
⠙ Preparing packages... (0/13)
graphql-relay ------------------------------ 14.86 KiB/16.54 KiB
uvicorn    ---------------

In [2]:
import mlflow
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

# Load Dataset

In [3]:
pwd

'/content'

In [24]:
# load the data
PATH = r"https://raw.githubusercontent.com/SagarNikam09/Influencer-Insights-Chrome-Plugin/refs/heads/main/cleaned_data.csv"
df = pd.read_csv(PATH)

df.head()

,comment,sentiment,stop_words_count,neg_stop_words_count,word_count,named_sentiment,sentence_length,response,nouns,verbs,adjectives
0,family mormon never tried explain still stare ...,1,13,0,39,positive,Long,0,16,4,2
1,buddhism much lot compatible christianity espe...,1,59,3,196,positive,Very Long,0,58,30,24
2,seriously say thing first get complex explain ...,-1,40,0,86,negative,Very Long,0,18,7,10
3,learned want teach different focus goal wrappi...,0,15,1,29,neutral,Long,0,7,4,2
4,benefit may want read living buddha living chr...,1,45,1,112,positive,Very Long,0,20,19,12


In [25]:
# Only use the comments column for now

final_df = df[['comment','named_sentiment']].copy()

final_df

,comment,named_sentiment
0,family mormon never tried explain still stare ...,positive
1,buddhism much lot compatible christianity espe...,positive
2,seriously say thing first get complex explain ...,negative
3,learned want teach different focus goal wrappi...,neutral
4,benefit may want read living buddha living chr...,positive
...,...,...
29746,hona hai vaccination education insurance end m...,neutral
29747,agree push make nation either pity pakistan in...,negative
29748,jesus,neutral
29749,downvote karna tha par upvote hogaya,neutral


In [26]:
# check for missing values

final_df.isna().sum()

,0
comment,180
named_sentiment,0


In [27]:
# missing values

final_df.loc[final_df['comment'].isna()]

,comment,named_sentiment
255,NaN,neutral
620,NaN,neutral
678,NaN,neutral
685,NaN,neutral
784,NaN,neutral
...,...,...
29411,NaN,neutral
29571,NaN,neutral
29598,NaN,neutral
29702,NaN,neutral


In [28]:
# remove the rows having missing values

print("Rows in data before removing missing values", final_df.shape[0])

final_df = final_df.dropna()

print("Rows in data after removing missing values", final_df.shape[0])

Rows in data before removing missing values 29751
Rows in data after removing missing values 29571


In [29]:
# check for duplicates

final_df.loc[final_df['comment'].duplicated(keep=False)].sort_values('comment')

,comment,named_sentiment
11011,aadhar,neutral
15254,aadhar,neutral
20346,aap,neutral
2346,aap,neutral
3110,aap,neutral
...,...,...
28187,yy,neutral
28172,yy,neutral
28081,yy,neutral
2537,zor bolo,neutral


In [30]:
# delete duplicates from data

final_df.drop_duplicates(inplace=True)

<ipython-input-30-bc10408d0ca5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.drop_duplicates(inplace=True)


In [31]:
final_df.isna().sum()

,0
comment,0
named_sentiment,0


# Experimentation

In [32]:
# make X and y

X = final_df['comment']
y = final_df['named_sentiment']
X

,comment
0,family mormon never tried explain still stare ...
1,buddhism much lot compatible christianity espe...
2,seriously say thing first get complex explain ...
3,learned want teach different focus goal wrappi...
4,benefit may want read living buddha living chr...
...,...
29745,let janta decide ulema cleric
29746,hona hai vaccination education insurance end m...
29747,agree push make nation either pity pakistan in...
29748,jesus


In [33]:
# do train test split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,
                                                    random_state=42, stratify=y)

print("The number of rows in train data are ",X_train.shape[0])
print("The number of rows in test data are ",X_test.shape[0])

The number of rows in train data are  23321
The number of rows in test data are  5831


In [34]:
# label encode the target

le = LabelEncoder()

y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [35]:
le.classes_

array(['negative', 'neutral', 'positive'], dtype=object)

In [36]:
from sklearn.metrics import accuracy_score, classification_report

In [37]:
# vectorize the the data
n_gram = (1,3)
max_features = 9000
vect = TfidfVectorizer(max_features=max_features, ngram_range=n_gram)

X_train_vec = vect.fit_transform(X_train)
X_test_vec = vect.transform(X_test)

In [38]:
# oversample the data
oversampler = SMOTE(random_state=42)

X_train_oversampled, y_train_oversampled = oversampler.fit_resample(X_train_vec,y_train)

In [39]:
print("The shape of X_train oversampled data is",X_train_oversampled.shape)

The shape of X_train oversampled data is (29022, 9000)


In [40]:
from collections import Counter

In [41]:
# value counts for the target

print(Counter(list(y_train_oversampled)))

Counter({np.int64(2): 9674, np.int64(0): 9674, np.int64(1): 9674})


In [42]:
# make a function to perform experimentation


def do_experimentation(classifier):
    clf = classifier
    # fit the pipeline on training data
    clf.fit(X_train_oversampled,y_train_oversampled)
    # calculate predictions
    y_pred = clf.predict(X_test_vec)

    with mlflow.start_run(run_name=f"{clf.__class__.__name__}") as run:
        # accuracy score
        accuracy = accuracy_score(y_test,y_pred)
        mlflow.log_metric("accuracy",accuracy)

        # classification report
        report = classification_report(y_test,y_pred,output_dict=True)
        # log classification report
        for label, metrics in report.items():
            if isinstance(metrics,dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}",value)

        # log the oversampling technique
        mlflow.log_param("classifier", clf.__class__.__name__)

    return "OK"

In [43]:
# set the tracking uri

mlflow.set_tracking_uri("http://ec2-18-212-49-1.compute-1.amazonaws.com:5000/")

In [44]:
# set experiment name

mlflow.set_experiment("Ex-5: Select the Classifier(Default Params)")

<Experiment: artifact_location='s3://yt-senti-mlflow/222591179857022594', creation_time=1745054642880, experiment_id='222591179857022594', last_update_time=1745054642880, lifecycle_stage='active', name='Ex-5: Select the Classifier(Default Params)', tags={}>

In [45]:
# run the experiment

models = [KNeighborsClassifier(n_jobs=-1),
          SVC(random_state=42),
          GradientBoostingClassifier(random_state=42),
          RandomForestClassifier(random_state=42,n_jobs=-1),
          LogisticRegression(random_state=42,n_jobs=-1),
          XGBClassifier(objective="multi:softmax",num_class=3),
          LGBMClassifier(objective="multiclass",num_class=3)]

for count, model in enumerate(models):
    print(f"Run Number {count+1}, STATUS: {do_experimentation(model)}")

🏃 View run KNeighborsClassifier at: http://ec2-18-212-49-1.compute-1.amazonaws.com:5000/#/experiments/222591179857022594/runs/5982529de68f4a54a16fc6c516f7590c
🧪 View experiment at: http://ec2-18-212-49-1.compute-1.amazonaws.com:5000/#/experiments/222591179857022594
Run Number 1, STATUS: OK
🏃 View run SVC at: http://ec2-18-212-49-1.compute-1.amazonaws.com:5000/#/experiments/222591179857022594/runs/f45e7470c21e408d8e74be3a5af52836
🧪 View experiment at: http://ec2-18-212-49-1.compute-1.amazonaws.com:5000/#/experiments/222591179857022594
Run Number 2, STATUS: OK
🏃 View run GradientBoostingClassifier at: http://ec2-18-212-49-1.compute-1.amazonaws.com:5000/#/experiments/222591179857022594/runs/31823b0aaed54ea9a0232fdea865488f
🧪 View experiment at: http://ec2-18-212-49-1.compute-1.amazonaws.com:5000/#/experiments/222591179857022594
Run Number 3, STATUS: OK
🏃 View run RandomForestClassifier at: http://ec2-18-212-49-1.compute-1.amazonaws.com:5000/#/experiments/222591179857022594/runs/5a2a807db6

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.969844 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 101827
[LightGBM] [Info] Number of data points in the train set: 29022, number of used features: 3534
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


🏃 View run LGBMClassifier at: http://ec2-18-212-49-1.compute-1.amazonaws.com:5000/#/experiments/222591179857022594/runs/fcac53d229b4471baacc56bcce67123f
🧪 View experiment at: http://ec2-18-212-49-1.compute-1.amazonaws.com:5000/#/experiments/222591179857022594
Run Number 7, STATUS: OK


**From the experiments `LighGBM` Classifier is giving good results**